In [ ]:
# @title Magic
%%capture
!pip install json-repair
!pip install -q evaluate
!pip install -q bert-score
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN_R')

!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-huggingface-api

!pip install -U transformers
!pip install -U bitsandbytes

!pip install pyarrow==15.0.0
!pip install datasets

import os
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.core import (
    VectorStoreIndex,
)

from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import SimilarityPostprocessor

import pandas as pd
import numpy as np
from collections import Counter
from datetime import date
import time
from time import sleep
import re
import json
from json_repair import repair_json
import ast
from evaluate import load
from google.colab import files

In [ ]:
# # @title [Llama] Setup model and Embedding
# %%capture
# !pip install llama-index
# !pip install llama-index-llms-huggingface
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-embeddings-huggingface-api
# import torch
# from llama_index.core import Settings
# from transformers import AutoTokenizer
# from llama_index.llms.huggingface import HuggingFaceLLM
# from transformers import BitsAndBytesConfig
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     token=hf_token,
# )

# stopping_ids = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>"),
# ]

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )


# # Get the model
# llm = HuggingFaceLLM(
#     model_name="swissnp/finetuned_gemini_CoT_studies",
#     tokenizer_name="swissnp/finetuned_gemini_CoT_studies",
#     model_kwargs={
#         "token": hf_token,
#         "quantization_config": quantization_config
#     },
#     context_window=8191,
#     max_new_tokens=2048,
#     generate_kwargs={
#         # "do_sample": True,
#         # # "temperature": 0.1,
#         # "top_p": 0.9,
#         'repetition_penalty': 1.175,
#         # 'early_stopping': True
#     },
#     # tokenizer_kwargs={"token": hf_token,}, #"max_length": 2048
#     stopping_ids=stopping_ids,
# )

# # Llama-3.1-8B-Instruct model
# Settings.llm = llm

# # bge embedding model
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = embed_model


In [ ]:
# # @title [Gemini] Setup model and Embedding
# !pip install llama-index-llms-gemini
# !pip install -q llama-index google-generativeai
# from llama_index.llms.gemini import Gemini

# Setup the model
# generation_config = {
#   "temperature": 0.1,
#   # "top_p": 1,
#   # "top_k": 1,
#   "max_output_tokens":8192,
# }

# safety_settings = [
#   {
#     "category": "HARM_CATEGORY_HARASSMENT",
#     "threshold": "BLOCK_NONE"
#   },
#   {
#     "category": "HARM_CATEGORY_HATE_SPEECH",
#     "threshold": "BLOCK_NONE"
#   },
#   {
#     "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
#     "threshold": "BLOCK_NONE"
#   },
#   {
#     "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
#     "threshold": "BLOCK_NONE"
#   },
# ]

# # Setup API_KEY
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# llm = Gemini(
#     model="models/gemini-1.5-flash-002",
#     # model="models/gemini-1.5-flash-8b",
#     # model="models/gemini-1.5-pro",
#     generation_config=generation_config,
#     safety_settings=safety_settings,
# )

# Settings.llm = llm

# embed_model_name = "BAAI/bge-small-en-v1.5"
# embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
# Settings.embed_model = embed_model

In [ ]:
# # @title Setup Query engine
# query_engine_get_study = CitationQueryEngine.from_args(
#     index_persisted,
#     similarity_top_k=20,
#     citation_chunk_size=2048,
#     verbose=True,
#     node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.8)],
# )

In [ ]:
# # @title Prompt ID06
# # query engine
# def query_response(study_information):
#   response = query_engine_get_study.query(f"""
#   Based on the provided instructions and clinical trial information, generate the new eligibility criteria specific for clinical trial information.

#   ### Instruction:
#   Find suitable papers that are relevant or similar to the provided clinical trial information (### Clinical Trial Information).
#   Prioritize the following topics when finding related studies:
#   1. Intervention
#   2. Study Design
#   3. Conditions
#   4. Study Description

#   Criteria Generation:
#   As a clinical researcher, generate new eligibility criteria for the given clinical trial information.
#   Analyze the information from all 20 related studies to generate new precise eligibility criteria.
#   Ensure that the criteria are specific for the given clinical trial information (### Clinical Trial Information).

#   Please follow the pattern of the output (### Pattern of the output).
#   --------------------------------------------------
#   ### Clinical Trial Information
#   {study_information}
#   --------------------------------------------------
#   ### Pattern of the Output
#   Inclusion Criteria
#   1.
#   2.
#   ...

#   Exclusion Criteria
#   1.
#   2.
#   ...
#   """
#   )
#   return response

In [ ]:
# # @title Generation
# print("Status: Generation")
# """"
# Prerequisites:
# 1. Define `citation_query_engine` and `query_response` functions, or edit the code in the generation section.
# 2. Upload file 'tags.csv'
# 3. Input: `method_used`

# Description:
# - Takes user input for the method name (`method_used`).
# - Retrieves the first 50 clinical trial entries from a dataset.
# - Iterates over these entries to query a response using `query_response()`.
# - Records the response, reference data, and the time taken for each query.
# - Saves the results into a CSV file and downloads it (if running in Google Colab).
# """
# # Get the method used as a string from user input
# method_used = str(input("Enter the method name used: "))

# def safe_json_load(json_string):
#     """
#     Safely loads a JSON string, attempting to repair it if necessary.
#     """
#     try:
#         # Check if the string is empty or whitespace only
#         if not json_string or json_string.isspace():
#             return {}

#         # Try to load JSON normally
#         return json.loads(json_string)
#     except (json.JSONDecodeError, TypeError, AttributeError):
#         # Attempt to repair JSON and reload
#         try:
#             repaired_string = repair_json(json_string)
#             return json.loads(repaired_string)
#         except (json.JSONDecodeError, TypeError, AttributeError):
#             # If repair fails, return an empty dictionary
#             return {}

# ## Get prompt for new dataset test
# df = dataset_test_df

# # Initialize empty lists to store results
# res_list = []
# ref_list = []
# time_taken_list = []

# # Iterate over the clinical trial info list
# for i in range(0, 206):
#   study_information = df.iloc[i]['data_prev']
#   max_retries = 5
#   retry_count = 0
#   success = False

#   while retry_count < max_retries and not success:
#     try:
#         start_time = time.time()
#         res = query_response(study_information)
#         end_time = time.time()
#         time_taken = end_time - start_time
#         time_taken_list.append(time_taken)
#         print(f"Row {i}, Time taken: {time_taken}s")
#         res_list.append(res.response)

#         # Store reference data
#         current_ref = [(node.node.metadata['NCT_ID'], node.score) for node in res.source_nodes]
#         ref_list.append(current_ref)

#         success = True  # Mark success to exit retry loop
#         # Pause before the next query
#         time.sleep(3)

#     except Exception as e:
#         retry_count += 1
#         print(f"Error processing row {i}, Attempt {retry_count}: {e}")
#         if retry_count >= max_retries:
#             print(f"Failed after {max_retries} attempts for row {i}")
#             res_list.append(None)
#             ref_list.append(None)
#             time_taken_list.append(None)
#         else:
#             time.sleep(3)
# # new dataset
# metadata_list = df['metadata'].tolist()
# data_list = df['data_new'].tolist()
# human_baseline_criteria = df['criteria_new'].tolist()

# # Create a zipped list
# zipped_criteria = list(zip(metadata_list, data_list, human_baseline_criteria, res_list, ref_list, time_taken_list))
# df_result = pd.DataFrame(zipped_criteria, columns=['metadata', 'data', 'human_baseline_criteria', method_used, 'source_nodes', 'time_taken'])

# # prev dataset
# data_list = df['data_prev'].tolist()
# human_baseline_criteria = df['criteria_prev'].tolist()

# # Create a zipped list
# zipped_criteria = list(zip(data_list, human_baseline_criteria, res_list, ref_list, time_taken_list))
# df_result = pd.DataFrame(zipped_criteria, columns=['data', 'human_baseline_criteria', method_used, 'source_nodes', 'time_taken'])

# # Generate filename with the current date
# today = date.today()
# date_str = today.strftime("%Y%m%d")
# file_name = f"{date_str}_{method_used}.csv"

# # Save the DataFrame to a CSV file and Download the saved CSV file (specific to Google Colab environment)
# df_result.to_csv(file_name, index=None)
# files.download(file_name)

# # Get Tags
# print("Status: Get Tags")
# """
# Description:
# - Retrieves clinical trial information, processes the dataset to clean the clinical trial criteria, and extracts tags.
# - Uses LLM to extract clinical trial-related tags based on eligibility criteria.
# - Saves the cleaned dataset and extracted tags into CSV files, which are then downloaded.
# """

# def process_dataframe(df, column_to_clean: str):
#     """
#     Cleans the specified column in the DataFrame by extracting the inclusion and exclusion criteria.
#     """
#     def clean_text(text):
#         return re.sub(r'[\#*]', '', str(text)).strip() if pd.notna(text) else ''

#     def extract_and_clean(text, pattern):
#         match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
#         return clean_text(match.group(1)) if match else ''

#     def get_exclusion_pattern(text):
#         return r'Exclusion Criteria(.*?)(?=Reference Papers)' if re.search(r'Reference Papers', text, re.IGNORECASE) else r'Exclusion Criteria(.*)'

#     def process_row(text):
#         if text is None or text.strip().lower() == 'empty response':
#             return ''

#         inclusion_pattern = r'Inclusion Criteria(.*?)(?=Exclusion Criteria)'
#         cleaned_inclusion = extract_and_clean(text, inclusion_pattern)
#         cleaned_exclusion = extract_and_clean(text, get_exclusion_pattern(text))

#         return (
#             "Inclusion Criteria:\n\n" + cleaned_inclusion + "\n\n" +
#             "Exclusion Criteria:\n\n" + cleaned_exclusion
#         )

#     df[f'cleaned_{column_to_clean}'] = df[column_to_clean].apply(process_row)

#     return df

# def safe_json_load(json_string):
#     """
#     Safely loads a JSON string, attempting to repair it if necessary.
#     """
#     try:
#         # Check if the string is empty or whitespace only
#         if not json_string or json_string.isspace():
#             return {}
#         # Try to load JSON normally
#         return json.loads(json_string)
#     except (json.JSONDecodeError, TypeError, AttributeError):
#         # Attempt to repair JSON and reload
#         try:
#             repaired_string = repair_json(json_string)
#             return json.loads(repaired_string)
#         except (json.JSONDecodeError, TypeError, AttributeError):
#             # If repair fails, return an empty dictionary
#             return {}

# def clean_json_string(input_string):
#     """
#     Cleans a JSON string by removing extraneous markdown-like markers.
#     """
#     input_string = str(input_string)
#     cleaned_string = input_string.replace("```json", "").replace("```", "")
#     return cleaned_string.strip()

# def get_tags(criteria, delay=3):
#     """
#     Extracts clinical trial-related tags from the given eligibility criteria.
#     """
#     if criteria is None:
#       return {}
#     prompt = f"""
# criteria = {criteria}

# As a clinical researcher, extract all clinical trial-related entity tags (e.g., bmi, nyha, sbp, cqt, life_expectancy, age) from the given eligibility criteria. Use consistent terms and abbreviations common in the clinical research field.

# Instructions:
# 1. Identify Relevant Entities:
# - Read through each criterion carefully.
# - Extract all relevant clinical entities.
# - Use standardized tags enclosed in angle brackets (e.g., <bmi>).

# 2. Maintain Consistency:
# - Use consistent tags for matching entities across all criteria.
# - Apply standard abbreviations common in clinical research.

# 3. Categorize Criteria:
# - Separate the entities into inclusion and exclusion categories based on where they appear in the criteria.

# 4. Output Format:
# - Present the results in JSON format with the following structure:
# {{
#   "inclusion": {{
#     "<tag>": "criterion"
#   }},
#   "exclusion": {{
#     "<tag>": "criterion"
#   }}
# }}

# Output Example:
# {{
#   "inclusion": {{
#     "<bmi>": "Body mass index (BMI) within the range of 19-35 kg/m² (inclusive)",
#     "<life_expectancy>": "Life expectancy of at least 12 weeks",
#     "<age>": "Age 60 years or older"
#   }},
#   "exclusion": {{
#     "<nyha>": "Serious dysrhythmias, obvious left ventricular dysfunction, New York Heart Association (NYHA) functional class III or IV",
#     "<sbp>": "Uncontrolled hypertension, systolic pressure ≥160 mmHg or diastolic pressure ≥100 mmHg",
#     "<cqt>": "Patients who have the second or third degree atrioventricular block, long Q-T syndrome, or QTc>500 ms without cardiac pacemaker"
#   }}
# }}

# Note: Do not provide extra ```json``` marks in the output.
# """
#     extraction_response = llm.complete(prompt)
#     time.sleep(delay)
#     cleaned_answer = clean_json_string(extraction_response)
#     tags_dict = safe_json_load(cleaned_answer)
#     return tags_dict

# process_dataframe(df_result, method_used)
# file_name_cleaned = file_name[:-4] + '_cleaned.csv'
# df_result.to_csv(file_name_cleaned, index=None)
# files.download(file_name_cleaned)

# df = pd.read_csv(file_name_cleaned)
# criteria_column = 'cleaned_' + method_used
# tags_dict_list = []

# for i in range(0,206):
#     start_time = time.time()
#     criteria = df.iloc[i][criteria_column]
#     tags_dict = get_tags(criteria)
#     end_time = time.time()
#     time_taken = end_time - start_time
#     print(f"Row {i}, Time taken: {time_taken}s")
#     tags_dict_list.append(tags_dict)

# column_name = method_used + '_tags'
# human_baseline_tags = pd.read_csv('tags.csv')
# tags_dict_list_str = [str(item) for item in tags_dict_list]
# new_tag_column = pd.DataFrame(tags_dict_list_str, columns=[column_name])
# new_tag_df = pd.concat([human_baseline_tags, new_tag_column], axis=1)

# file_name_tag = file_name[:-4] + '_tags.csv'
# new_tag_df.to_csv(file_name_tag, index=False)
# files.download(file_name_tag)

Enter the method name used: prev_dataset_2


In [7]:
# @title Evaluation
import pandas as pd
import numpy as np
from evaluate import load
from collections import Counter
import ast

print("Status: Evaluation")
"""
Description:
This script performs an evaluation of generated tags against a human baseline by:
- Using BERTScore to compare inclusion and exclusion tags.
"""

bertscore = load("bertscore")

def convert_str_tags_to_dict(df, column_names):
    """
    Converts string representations of dictionaries in specified columns to actual Python dictionaries.
    Skips empty strings and handles invalid values gracefully.
    """
    def safe_eval(x):
        if pd.isna(x) or x == '':
            return {}
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return {}

    for column in column_names:
        df[column] = df[column].apply(safe_eval)

    return df

def prepare_tag_texts_and_count(df, reference_col, prediction_col, tag_type):
    """
    Prepare tag texts for BERTScore evaluation and count matching tags
    tag_texts = {'<tag>': {'references': [], 'predictions': []}, , , ,}
    """
    tag_texts = {}
    counter = Counter()
    for _, row in df.iterrows():
        ref_dict = row[reference_col].get(tag_type, {})
        pred_dict = row[prediction_col].get(tag_type, {})

        for tag, ref_text in ref_dict.items():
            tag = tag.lower()
            if tag in pred_dict:
                if tag not in tag_texts:
                    tag_texts[tag] = {'references': [], 'predictions': []}
                tag_texts[tag]['references'].append(ref_text)
                tag_texts[tag]['predictions'].append(pred_dict[tag])
                counter[tag] += 1

    return tag_texts, counter

def compute_bertscore_for_criteria(tag_texts, tag_counter, criteria_type):
    """This function computes BERTScore for given tag texts and return the results."""
    results = {}
    for tag, texts in tag_texts.items():
        count = tag_counter[tag]
        references = texts['references']
        predictions = texts['predictions']
        if references and predictions:
            score = bertscore.compute(
                predictions=predictions,
                references=references,
                lang='en',
                model_type="distilbert-base-uncased"
            )
            avg_f1 = sum(score['f1']) / len(score['f1'])
            results[tag] = {
                'count': count,
                'bertscore': score,
                'avg_f1': avg_f1
            }
    return results

def compute_avg_scores_for_each_tag(tag_results):
    """This function calculates average scores for each tag and returns a DataFrame."""
    data = []
    for tag, result in tag_results.items():
        if 'bertscore' in result:
            data.append({
                'Tag': tag,
                'Count': result['count'],
                'Avg_BERTScore_Precision': np.mean(result['bertscore']['precision']),
                'Avg_BERTScore_Recall': np.mean(result['bertscore']['recall']),
                'Avg_BERTScore_F1': result['avg_f1']
            })
    return pd.DataFrame(data)

def compute_avg_scores_for_tag_type(tag_results):
    """Calculate the overall average precision, recall, and F1 score for a tag type."""
    precisions = [np.mean(result['bertscore']['precision']) for result in tag_results.values() if 'bertscore' in result]
    recalls = [np.mean(result['bertscore']['recall']) for result in tag_results.values() if 'bertscore' in result]
    f1_scores = [result['avg_f1'] for result in tag_results.values() if 'avg_f1' in result]

    avg_precision = np.mean(precisions) if precisions else None
    avg_recall = np.mean(recalls) if recalls else None
    avg_f1 = np.mean(f1_scores) if f1_scores else None

    return avg_precision, avg_recall, avg_f1

def compute_bertscore_for_tags(df, reference_col, prediction_cols):
    results = {}

    for prediction_col in prediction_cols:
        results[prediction_col] = {'inclusion': {}, 'exclusion': {}}

        for tag_type in ['inclusion', 'exclusion']:
            true_positives = 0
            false_positives = 0
            false_negatives = 0

            for _, row in df.iterrows():
                ref_tags = set(row[reference_col].get(tag_type, {}).keys())
                pred_tags = set(row[prediction_col].get(tag_type, {}).keys())

                true_positives += len(ref_tags & pred_tags)
                false_positives += len(pred_tags - ref_tags)
                false_negatives += len(ref_tags - pred_tags)

            precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
            recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            results[prediction_col][tag_type] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'true_positives': true_positives,
                'false_positives': false_positives,
                'false_negatives': false_negatives
            }

    return results

def evaluate_tag_pairs(df, reference_col, prediction_cols, tag_types):
    """This function evaluates tag pairs, computes BERTScores, and saves results to CSV files."""
    data = []
    for prediction_col in prediction_cols:
        for tag_type in tag_types:
            tag_texts, tag_counter = prepare_tag_texts_and_count(df, reference_col, prediction_col, tag_type)
            criteria_bert_results = compute_bertscore_for_criteria(tag_texts, tag_counter, tag_type)
            scores_df = compute_avg_scores_for_each_tag(criteria_bert_results)
            file_name_tag_type = f'{tag_type}_{prediction_col}_vs_{reference_col}_bertscore.csv'
            scores_df.to_csv(file_name_tag_type, index=False)
            # files.download(file_name_tag_type)
            avg_precision, avg_recall, avg_f1 = compute_avg_scores_for_tag_type(criteria_bert_results)

            tag_bert_result = compute_bertscore_for_tags(df, reference_col, prediction_cols)
            precision = tag_bert_result[prediction_col][tag_type]['precision']
            recall = tag_bert_result[prediction_col][tag_type]['recall']
            f1 = tag_bert_result[prediction_col][tag_type]['f1']

            data.append({
                'Prediction_Column': prediction_col,
                'Tag_Type': tag_type,
                'Criteria_Avg_Precision': avg_precision,
                'Criteria_Avg_Recall': avg_recall,
                'Criteria_Avg_F1': avg_f1,
                'Tag_Avg_Precision': precision,
                'Tag_Avg_Recall': recall,
                'Tag_Avg_F1': f1
            })

    # Convert data to DataFrame and save to CSV
    results_df = pd.DataFrame(data)
    file_name_eval = method_used + '_evaluation_summary.csv'
    results_df.to_csv(file_name_eval, index=False)
    # files.download(file_name_eval)

# Main execution
if __name__ == "__main__":
    # Load the DataFrame
    tag_df = pd.read_csv(file_name_tag)

    # Convert string values to dictionaries
    columns_to_convert = ['human_baseline_tags', column_name]
    tag_df = convert_str_tags_to_dict(tag_df, columns_to_convert)

    # Define columns and tag types
    reference_col = 'human_baseline_tags'
    prediction_cols = [column_name]
    tag_types = ['inclusion', 'exclusion']

    # Run the evaluation
    evaluate_tag_pairs(tag_df, reference_col, prediction_cols, tag_types)

ModuleNotFoundError: No module named 'evaluate'

In [4]:
## load the df_result_gemini_18_nov_1154.pkl to df
import pandas as pd
# df_info = pd.read_pickle('gemini_CoT_for_test_dataset.pkl')
df = pd.read_pickle('df_result_gemini_19_nov_0200.pkl')

# merge the df_info and df based on input
# df = pd.merge(df_info, df, on='input', how='inner') 
# filter only the one that include </Criteria> in the output
df = df[df['output'].str.contains('</CRITERIA>')]
# df.drop(columns=['output_x'], inplace=True)
# df.rename(columns={'output_y': 'output'}, inplace=True)
df = df.reset_index(drop=True)
df
df.head()




,index,encoded_related_studies,title,description,desired_criteria,messages,response,input,output,input_len
0,1,<STUDY>\nExample Title: An Open Registry to Me...,An Open Registry to Measure the Impact of Addi...,#Study Description \nBrief Summary \nThis regi...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Defi...,2828
1,2,"<STUDY>\nExample Title: An Open-Label, Multice...","A Phase I, Open-Label, Dose-Escalation Study o...",#Study Description \nBrief Summary \nThis is a...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Unde...,3248
2,4,<STUDY>\nExample Title: REal-world Patterns of...,Treatment Patterns And Clinical Outcomes Among...,#Study Description \nBrief Summary \nCDK4/6 in...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: REal-...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: REa...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Defi...,3345
3,6,<STUDY>\nExample Title: HI-CHART: A Phase I/II...,A Phase I/II Trial of Isotoxic Accelerated Rad...,#Study Description \nBrief Summary \nThe I-STA...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: HI-CH...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: HI-...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Unde...,5917
4,7,<STUDY>\nExample Title: Resistance Training an...,Exercise and Nutrition for Head and Neck Cance...,#Study Description \nBrief Summary \nResearch ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Resis...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: Res...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Esta...,4460


In [5]:
import re

def extract_criteria(text):
    match = re.search(r"<CRITERIA>(.*?)</CRITERIA>", text, re.DOTALL)
    return match.group(1).strip() if match else None

# Update the parsing function to include raw <CRITERIA> content in the JSON
def improved_parse_with_raw(criteria_text):
    criteria_text = extract_criteria(criteria_text)
    if not criteria_text:
        print("No criteria found")
        return {}
    result = {
        "raw_criteria": criteria_text,  # Include raw content for debugging
        "inclusion_criteria": [],
        "exclusion_criteria": [],
        "sex": "ALL",
        "ages": {
            "minimum_age": None,
            "maximum_age": None,
            "age_group": []
        },
        "accepts_healthy_volunteers": False
    }
    
    # Extract inclusion and exclusion criteria
    inclusion_match = re.search(r"Inclusion Criteria:(.*?)(?:Exclusion Criteria:|##|$)", criteria_text, re.DOTALL)
    exclusion_match = re.search(r"Exclusion Criteria:(.*?)(?:##|$)", criteria_text, re.DOTALL)

    if inclusion_match:
        result["inclusion_criteria"] = [
            item.strip() for item in inclusion_match.group(1).split("\n") if item.strip()
        ]
    if exclusion_match:
        result["exclusion_criteria"] = [
            item.strip() for item in exclusion_match.group(1).split("\n") if item.strip()
        ]

    # Extract demographic information
    sex_match = re.search(r"##Sex\s*:\s*(Male|Female|All)", criteria_text, re.IGNORECASE)
    if sex_match:
        result["sex"] = sex_match.group(1).upper()

    min_age_match = re.search(r"- Minimum Age\s*:\s*(\d+)", criteria_text, re.IGNORECASE)
    if min_age_match:
        result["ages"]["minimum_age"] = int(min_age_match.group(1))

    max_age_match = re.search(r"- Maximum Age\s*:\s*(\d+)", criteria_text, re.IGNORECASE)
    if max_age_match:
        result["ages"]["maximum_age"] = int(max_age_match.group(1))

    age_group_match = re.findall(r"Age Group.*?:(.*?)$", criteria_text, re.MULTILINE)
    if age_group_match:
        age_groups = re.findall(r"(Child|Adult|Older Adult)", " ".join(age_group_match), re.IGNORECASE)
        result["ages"]["age_group"] = list(set(group.upper() for group in age_groups))  # Unique values

    # Check for healthy volunteers
    healthy_volunteers_match = re.search(r"##Accepts Healthy Volunteers:\s*(Yes|No)", criteria_text, re.IGNORECASE)
    if healthy_volunteers_match:
        result["accepts_healthy_volunteers"] = healthy_volunteers_match.group(1).strip().lower() == "yes"

    return result

df.dropna(subset=['output'], inplace=True)
# Re-parse the criteria content with raw content included
improved_criteria_with_raw_json = df['output'].apply(improved_parse_with_raw)
# add the improved_criteria_with_raw_json to the df as a new column named "json"

df['json'] = improved_criteria_with_raw_json


In [6]:
df

,index,encoded_related_studies,title,description,desired_criteria,messages,response,input,output,input_len,json
0,1,<STUDY>\nExample Title: An Open Registry to Me...,An Open Registry to Measure the Impact of Addi...,#Study Description \nBrief Summary \nThis regi...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Defi...,2828,{'raw_criteria': '#Eligibility Criteria: Inclu...
1,2,"<STUDY>\nExample Title: An Open-Label, Multice...","A Phase I, Open-Label, Dose-Escalation Study o...",#Study Description \nBrief Summary \nThis is a...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: An Op...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: An ...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Unde...,3248,{'raw_criteria': '#Eligibility Criteria: Inclu...
2,4,<STUDY>\nExample Title: REal-world Patterns of...,Treatment Patterns And Clinical Outcomes Among...,#Study Description \nBrief Summary \nCDK4/6 in...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: REal-...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: REa...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Defi...,3345,{'raw_criteria': '#Eligibility Criteria: Inclu...
3,6,<STUDY>\nExample Title: HI-CHART: A Phase I/II...,A Phase I/II Trial of Isotoxic Accelerated Rad...,#Study Description \nBrief Summary \nThe I-STA...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: HI-CH...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: HI-...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Unde...,5917,{'raw_criteria': '#Eligibility Criteria: Inclu...
4,7,<STUDY>\nExample Title: Resistance Training an...,Exercise and Nutrition for Head and Neck Cance...,#Study Description \nBrief Summary \nResearch ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Resis...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: Res...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Esta...,4460,{'raw_criteria': '#Eligibility Criteria: Inclu...
...,...,...,...,...,...,...,...,...,...,...,...
2598,3988,"<STUDY>\nExample Title: A Phase 1, Open-label,...","A Phase 1, Multiple-Dose Study of the Safety a...",#Study Description \nBrief Summary \nThe purpo...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: A Pha...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: A P...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Unde...,3432,{'raw_criteria': '#Eligibility Criteria: Inclu...
2599,3989,<STUDY>\nExample Title: The Impact of Preopera...,Do Omega-3 Fatty Acids Have Any Impact On Seru...,#Study Description \nBrief Summary \nPre- and ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: The I...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: The...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Unde...,5480,{'raw_criteria': '#Eligibility Criteria: Inclu...
2600,3990,<STUDY>\nExample Title: Aerobic and Resistance...,Effects of Water-based Versus Land-based Exerc...,#Study Description \nBrief Summary \nPhysical ...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Aerob...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: Aer...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Defi...,4921,{'raw_criteria': '#Eligibility Criteria: Inclu...
2601,3991,<STUDY>\nExample Title: Hepassocin Levels in P...,Clusterin Level Determination and Its Associat...,#Study Description \nBrief Summary \nClusterin...,#Eligibility Criteria:\nInclusion Criteria:\n\...,<EXAMPLE_STUDIES><STUDY>\nExample Title: Hepas...,NaN,<RELATED_STUDIES>\n<STUDY>\nExample Title: Hep...,<STEP-BY-STEP-JUSTIFICATION>\n\n**Step 1: Defi...,3159,{'raw_criteria': '#Eligibility Criteria: Inclu...
